In [2]:
import pandas as pd
import tensorflow
import numpy as np
from numpy import exp 
from numpy import transpose as tp
import sklearn
from sklearn import preprocessing
import random

In [34]:

class Dense:
    def __init__(self,neurons,i):
        self.neurons = neurons
        self.layernumber = i
        
    def MatrixInitialize(self,neurlist,inpshape): 
        if self.layernumber == 0:
            self.mat = abs(np.random.randn(self.neurons,inpshape)*np.sqrt(1/4))
        else:
            self.mat = abs(np.random.randn(self.neurons,neur[self.layernumber-1])*np.sqrt(1/4)+1)
            
    def BiasInitialize(self,neurlist):
        if self.layernumber != 0  :
            self.bias = np.random.randn(neur[self.layernumber],1)
        else:
            self.bias = np.zeros((neur[self.layernumber],1))
            
    def Matrix(self,ins):
        return self.mat@ins
    
    def ReLu(self,ins):
        return ins * (ins > 0)
        #return 1/(1+np.exp(-ins))
    
    def DerReLu(self,ins):
        return np.diag(np.ndarray.flatten((ins>=0)*1))
        #return np.diag(np.ndarray.flatten(np.divide(exp(-ins),(1+exp(-ins))**2)))
    
    def DerReLuLayer(self,inp):
        return self.DerReLu(self.Matrix(inp)+self.bias)

            
    def GradientProp(self,inp):
        return self.DerReLuLayer(inp)@self.mat
    
    def LayerOut(self,ins):
        ins = self.Matrix(ins)+self.bias
        return self.ReLu(ins)

        
def Setup(neur,inpshape):    
    classlist = []
    for i in range(len(neur)):
        classlist.append(Dense(neur[i],i))
        classlist[i].MatrixInitialize(neur,inpshape)
        classlist[i].BiasInitialize(neur)
    return classlist


In [35]:
def NetworkPass(classlist,inp1): #One pass through the network with 1 instance
    n_samples = inp.shape[0]
    for i in range(len(classlist)):
        pass_out = classlist[i].LayerOut(inp1)
        inp1 = pass_out
    return pass_out

def NetworkError(inp,out):
    n_samples = inp.shape[0]
    e_array = []
    for k in range(n_samples):
        inp1 = inp[k]
        out1 = out[k,:]
        out_e = NetworkPass(classlist,inp1)   
        e_array.append(float(out1 - out_e))
    e_array = np.asarray(e_array)
    mse = np.dot(e_array,e_array)
    return e_array,mse

def LayerInputs(inp):
    inplist=[inp]
    for i in range(len(classlist)):
        if i != max(range(len(classlist))):
            inp = classlist[i].LayerOut(inp)
            inplist.append(inp)
    return inplist
def GradientChain(inp):
    #inp = layer_in
    #mult = 0
    #for i in range(layer_no + 1,len(classlist)):
    #    layer_out = classlist[i-1].LayerOut(inp)
    #    local_der = classlist[i].GradientProp(layer_out)
    #    if i == layer_no +1 :
    #        mult = local_der
    #    else:
    #        mult = local_der@mult
    #    inp = layer_out
        #print(layer_out)
    #if layer_no+1 == len(classlist):
    #    mult = np.identity(classlist[layer_no].neurons)
    #return mult 
    dProp = []
    dRelu = []
    chain = []
    for i in range(len(classlist)):
        dPr = classlist[i].GradientProp(inp[i])
        dRe = classlist[i].DerReLuLayer(inp[i])
        dProp.append(dPr)
        dRelu.append(dRe)
    for i in range(len(classlist)-1):
        mult = np.identity(dProp[i+1].shape[1])
        for j in range(i+1,len(classlist)):
            mult = dProp[j]@mult
        mult = mult@dRelu[i]
        chain.append(mult)
    chain.append(dRelu[-1])
    return chain

In [39]:


def NetworkGradient(classlist,inp,out):
    storage = []
    dif,mse = NetworkError(inp,out)
    for k in range(inp.shape[0]):
        inplist= LayerInputs(inp[k])
        gchain = GradientChain(inplist)
        for i in range(len(classlist)):
            layer_inp = inplist[i] 
            [row_no,column_no] = classlist[i].mat.shape
            dmat = np.zeros((row_no,column_no+1))
            for j in range(column_no+1):
                if j != column_no:
                    id_input = (np.identity(row_no)*float(layer_inp[j]))
                else:
                    id_input = (np.identity(row_no))
                dodj = gchain[i]@id_input                      
                dedj = (-2*dif[k]*dodj)
                dmat[:,j] = dedj
            if k == 0:
                storage.append(dmat) 
            else:
                storage[i] += dmat
    return storage
            


In [37]:
u =np.asarray(pd.read_csv('in.dat',sep = ',',header = None))
scaler = preprocessing.MinMaxScaler()
u = scaler.fit_transform(u)
inp = np.reshape(u,(u.shape[0],u.shape[1],1))

y = np.reshape(np.asarray(pd.read_csv('out.dat',sep=',',header = None)),(-1,1))
scaler2 = preprocessing.MinMaxScaler()
out = scaler2.fit_transform(y)

neur = [4,1]
batch_size = 1000
input_dim = inp.shape[1]
classlist = Setup(neur,input_dim)

In [41]:

mse = 1
while mse > 1e-3:    
        st = []
        st = NetworkGradient(classlist,inp,out)
        for i in range(len(classlist)):
            classlist[i].mat = classlist[i].mat - 0.0001*st[i][:,0:-1]
            if i != 0:
                classlist[i].bias = classlist[i].bias - 0.0001*np.reshape(st[i][0,-1],(-1,1))
        su,ms = NetworkError(inp,out)
        print('MSE Loss: %.6f'%(ms))
        print(np.linalg.norm(st[0]),np.linalg.norm(st[1]))#,np.linalg.norm(st[2]))
        

MSE Loss: 0.149152
0.3543704051777191 0.3488083531606652
MSE Loss: 0.149139
0.35393312947124156 0.34841579880562334
MSE Loss: 0.149125
0.3534963575981461 0.34802368243404147
MSE Loss: 0.149111
0.3530600889886619 0.347632003566458
MSE Loss: 0.149097
0.35262432307367286 0.3472407617238651
MSE Loss: 0.149083
0.35218905928477806 0.34684995642774746
MSE Loss: 0.149069
0.3517542970542785 0.34645958720007547
MSE Loss: 0.149055
0.3513200358151456 0.34606965356328256
MSE Loss: 0.149042
0.35088627500108804 0.3456801550403132
MSE Loss: 0.149028
0.35045301404646984 0.3452910911545646
MSE Loss: 0.149014
0.3500202523863842 0.34490246142994024
MSE Loss: 0.149001
0.34958798945657504 0.34451426539079505
MSE Loss: 0.148987
0.3491562246935034 0.34412650256198113
MSE Loss: 0.148973
0.3487249575343181 0.34373917246882896
MSE Loss: 0.148960
0.3482941874168507 0.34335227463714246
MSE Loss: 0.148946
0.34786391377963044 0.34296580859321174
MSE Loss: 0.148933
0.3474341360618745 0.34257977386380567
MSE Loss: 0.1

MSE Loss: 0.146699
0.40691531791981944 0.39225820591425514
MSE Loss: 0.146682
0.40631065219701273 0.39172112326054215
MSE Loss: 0.146665
0.40570726581577166 0.3911850418879429
MSE Loss: 0.146648
0.40510515266666225 0.39064995816595743
MSE Loss: 0.146631
0.40450430680259947 0.3901158685415231
MSE Loss: 0.146614
0.40390472242131015 0.38958276953083393
MSE Loss: 0.146597
0.4033063938501156 0.3890506577122358
MSE Loss: 0.146580
0.4027093155326212 0.38851952971999576
MSE Loss: 0.146564
0.4021134820172317 0.38798938223893065
MSE Loss: 0.146547
0.4015188879470234 0.38746021199965935
MSE Loss: 0.146530
0.40092552805102416 0.38693201577450587
MSE Loss: 0.146514
0.4003333971365873 0.38640479037392433
MSE Loss: 0.146497
0.39974249008269463 0.3858785326433427
MSE Loss: 0.146481
0.39915280183415436 0.38535323946043515
MSE Loss: 0.146464
0.3985643273965875 0.3848289077327567
MSE Loss: 0.146448
0.39797706183193315 0.38430553439562637
MSE Loss: 0.146432
0.3973910002545995 0.38378311641030227
MSE Loss:

MSE Loss: 0.144648
0.3304342852906123 0.3234628305568108
MSE Loss: 0.144636
0.32997295434634116 0.3230437939640699
MSE Loss: 0.144624
0.3295123925691805 0.3226254214429924
MSE Loss: 0.144613
0.3290525977055931 0.3222077113504839
MSE Loss: 0.144601
0.3285935675137831 0.3217906620501681
MSE Loss: 0.144590
0.3281352997635875 0.3213742719123372
MSE Loss: 0.144578
0.3276777922364434 0.320958539313936
MSE Loss: 0.144567
0.327221042725248 0.3205434626384919
MSE Loss: 0.144555
0.3267650490343395 0.3201290402761087
MSE Loss: 0.144544
0.32630980897941714 0.3197152706234282
MSE Loss: 0.144533
0.3258553203874356 0.3193021520835759
MSE Loss: 0.144521
0.3254015810965655 0.31888968306614623
MSE Loss: 0.144510
0.32494858895607925 0.318477861987144
MSE Loss: 0.144499
0.3244963418263364 0.3180666872689809
MSE Loss: 0.144487
0.32404483757867303 0.3176561573404189
MSE Loss: 0.144476
0.3235940740953292 0.3172462706365363
MSE Loss: 0.144465
0.3231440492694118 0.3168370255987128
MSE Loss: 0.144454
0.32269476

MSE Loss: 0.143220
0.27025576903833004 0.2685833058804117
MSE Loss: 0.143212
0.26988768454929357 0.2682466774414876
MSE Loss: 0.143204
0.2695201334276659 0.26791052907125756
MSE Loss: 0.143196
0.2691531145347774 0.26757485981136403
MSE Loss: 0.143187
0.26878662673666487 0.2672396687066384
MSE Loss: 0.143179
0.2684206689039768 0.26690495480504295
MSE Loss: 0.143171
0.2680552399119284 0.26657071715764247
MSE Loss: 0.143163
0.26769033864037955 0.26623695481865983
MSE Loss: 0.143155
0.2673259639737028 0.26590366684539113
MSE Loss: 0.143147
0.26696211480079673 0.2655708522982206
MSE Loss: 0.143139
0.2665987900150578 0.2652385102406008
MSE Loss: 0.143132
0.2662359885143555 0.2649066397390419
MSE Loss: 0.143124
0.2658737092009921 0.26457523986308923
MSE Loss: 0.143116
0.26551195098170455 0.26424430968532325
MSE Loss: 0.143108
0.26515071276759494 0.263913848281321
MSE Loss: 0.143100
0.2647899934741382 0.2635838547296617
MSE Loss: 0.143092
0.2644297920211539 0.26325432811191235
MSE Loss: 0.1430

MSE Loss: 0.142217
0.22165486115528843 0.22408065339658753
MSE Loss: 0.142211
0.22135454014188632 0.22380533942140796
MSE Loss: 0.142205
0.2210546242586133 0.223530392666402
MSE Loss: 0.142200
0.22075511282821653 0.2232558125098801
MSE Loss: 0.142194
0.22045600517550348 0.2229815983317967
MSE Loss: 0.142188
0.2201573006273583 0.22270774951376468
MSE Loss: 0.142183
0.21985899851266488 0.22243426543899866
MSE Loss: 0.142177
0.21956109816236277 0.22216114549235527
MSE Loss: 0.142171
0.2192635989094164 0.22188838906031494
MSE Loss: 0.142166
0.2189665000887926 0.22161599553096387
MSE Loss: 0.142160
0.21866980103746286 0.2213439642939972
MSE Loss: 0.142154
0.21837350109438086 0.2210722947407027
MSE Loss: 0.142149
0.2180775996004995 0.22080098626397582
MSE Loss: 0.142143
0.21778209589872485 0.22053003825828552
MSE Loss: 0.142138
0.2174869893339358 0.22025945011968823
MSE Loss: 0.142132
0.21719227925294177 0.21998922124580012
MSE Loss: 0.142127
0.21689796500453704 0.21971935103583723
MSE Loss:

...

MSE Loss: 0.137216
0.11525256895277415 0.03716850498253995
MSE Loss: 0.137215
0.11523287708760444 0.0371560478302692
MSE Loss: 0.137215
0.11521318116280671 0.03714360976720555
MSE Loss: 0.137214
0.11519348118213091 0.03713119074663991
MSE Loss: 0.137213
0.11517377714945894 0.03711879072180044
MSE Loss: 0.137213
0.11515406906855406 0.03710640964618452
MSE Loss: 0.137212
0.11513435694330199 0.03709404747323738
MSE Loss: 0.137211
0.11511464077755537 0.03708170415656203
MSE Loss: 0.137211
0.11509492057521194 0.03706937964980787
MSE Loss: 0.137210
0.11507519634017704 0.03705707390672855
MSE Loss: 0.137209
0.11505546807631446 0.037044786881244456
MSE Loss: 0.137209
0.11503573578754435 0.03703251852730984
MSE Loss: 0.137208
0.11501599947784791 0.037020268798906505
MSE Loss: 0.137208
0.11499625915116793 0.03700803765017977
MSE Loss: 0.137207
0.11497651481146152 0.036995825035366414
MSE Loss: 0.137206
0.1149567664626998 0.03698363090878912
MSE Loss: 0.137206
0.11493701410888768 0.03697145522484

MSE Loss: 0.137128
0.11247832975633411 0.035600878789802504
MSE Loss: 0.137128
0.11245811577465487 0.0355906649462508
MSE Loss: 0.137127
0.11243789837977716 0.03558046452251329
MSE Loss: 0.137126
0.11241767757700452 0.03557027748355755
MSE Loss: 0.137126
0.11239745337168607 0.03556010379437725
MSE Loss: 0.137125
0.11237722576914626 0.03554994342008774
MSE Loss: 0.137125
0.11235699477471672 0.03553979632587287
MSE Loss: 0.137124
0.11233676039372069 0.035529662477023494
MSE Loss: 0.137123
0.11231652263152672 0.03551954183884652
MSE Loss: 0.137123
0.11229628149351922 0.03550943437672177
MSE Loss: 0.137122
0.11227603698503988 0.03549934005616112
MSE Loss: 0.137122
0.1122557891114751 0.035489258842712694
MSE Loss: 0.137121
0.11223553787817406 0.03547919070205093
MSE Loss: 0.137120
0.11221528329054797 0.03546913559985546
MSE Loss: 0.137120
0.11219502535398485 0.03545909350191918
MSE Loss: 0.137119
0.11217476407387829 0.035449064374115796
MSE Loss: 0.137119
0.11215449945563571 0.0354390481823

MSE Loss: 0.137047
0.10961763103168627 0.03428695922376276
MSE Loss: 0.137047
0.1095969934189023 0.034278315793404096
MSE Loss: 0.137046
0.10957635318685889 0.03426968157320246
MSE Loss: 0.137046
0.10955571034141509 0.03426105653782954
MSE Loss: 0.137045
0.10953506488850238 0.03425244066193145
MSE Loss: 0.137044
0.10951441683401864 0.034243833920261556
MSE Loss: 0.137044
0.10949376618385442 0.03423523628764599
MSE Loss: 0.137043
0.10947311294397437 0.03422664773887532
MSE Loss: 0.137043
0.10945245712027463 0.03421806824889437
MSE Loss: 0.137042
0.10943179871866761 0.03420949779268587
MSE Loss: 0.137042
0.10941113774508393 0.03420093634527681
MSE Loss: 0.137041
0.10939047420544451 0.0341923838817657
MSE Loss: 0.137041
0.10936980810565192 0.03418384037733791
MSE Loss: 0.137040
0.10934913945166114 0.03417530580717207
MSE Loss: 0.137039
0.10932846824937091 0.034166780146583656
MSE Loss: 0.137039
0.10930779450471315 0.03415826337090745
MSE Loss: 0.137038
0.10928711822363443 0.03414975545551

MSE Loss: 0.136973
0.10670554065182238 0.03315602283816159
MSE Loss: 0.136972
0.10668459423892107 0.033148447508972466
MSE Loss: 0.136972
0.10666364604026154 0.03314087838216095
MSE Loss: 0.136971
0.10664269606172799 0.033133315439916
MSE Loss: 0.136971
0.10662174430930714 0.03312575866433446
MSE Loss: 0.136970
0.10660079078887119 0.03311820803771019
MSE Loss: 0.136970
0.10657983550635085 0.033110663542306684
MSE Loss: 0.136969
0.10655887846768232 0.03310312516041833
MSE Loss: 0.136969
0.10653791967879493 0.03309559287440502
MSE Loss: 0.136968
0.10651695914558158 0.03308806666670524
MSE Loss: 0.136968
0.10649599687398639 0.03308054651974113
MSE Loss: 0.136967
0.10647503286991732 0.033073032416026334
MSE Loss: 0.136967
0.10645406713927352 0.03306552433812832
MSE Loss: 0.136966
0.10643309968801422 0.03305802226858293
MSE Loss: 0.136966
0.10641213052200534 0.03305052619008487
MSE Loss: 0.136965
0.10639115964716055 0.03304303608531609
MSE Loss: 0.136965
0.10637018706940685 0.03303555193698

KeyboardInterrupt: 

In [42]:
inst = 5
pred = NetworkPass(classlist,inp[inst])
print(out[inst])
print(pred)

[0.78029052]
[[0.71367153]]


(array([-4.50774333, -2.26300266, -4.90950238, -3.17158372, -4.55235584,
        -3.19356075, -3.71704902, -1.86434311, -1.88431471, -2.37378276,
        -3.32612969, -2.6712802 , -3.87406034, -2.20476724, -2.89406687,
        -3.192989  , -3.76603787, -1.75704422, -2.95488885, -2.05642545,
        -3.5159898 , -3.74178579, -1.76355188, -3.18551409, -2.31463537,
        -2.89036061, -2.70789058, -2.94433925, -3.53995695, -3.47253731,
        -3.19693578, -3.58132095, -3.76320972, -1.06809339, -3.07064703,
        -3.61975464, -1.09245401, -1.09927058, -4.33497694, -2.67213961,
        -2.79403252, -2.61105343, -1.73030553, -1.46775877, -3.66418929,
        -2.283286  , -2.72729111, -3.27246497, -2.46622566, -3.71612786,
        -3.23377386, -3.89530504, -2.74252439, -2.10849952, -1.55964477,
        -2.85658099, -3.68366429, -2.08948224, -2.62177506, -2.26820584,
        -1.21747813, -4.13833916, -2.88022016, -2.68098534, -3.70042793,
        -3.70068284, -2.03384461, -2.73415329, -4.2

array([[[0.62275055],
        [0.39505279],
        [0.35870833],
        [0.08900082]],

       [[0.74147438],
        [0.6947723 ],
        [0.82407557],
        [0.83209404]],

       [[0.9268783 ],
        [0.74112534],
        [0.56711235],
        [0.97364939]],

       [[0.23497213],
        [0.52996821],
        [0.05048864],
        [0.7606126 ]]])

In [ ]:
u = np.asmatrix(pd.read_csv('in.dat',sep = ',',header = None))
y = np.asmatrix(pd.read_csv('out.dat',sep=',',header = None))
scaler = preprocessing.MinMaxScaler()
u = np.asmatrix(scaler.fit_transform(u))
y = np.reshape(np.asmatrix(scaler.fit_transform(np.reshape(y,(100,1)))),(1,100))

In [ ]:
w1 = np.random.randn(4,4)*np.sqrt(1/4)
w2 = np.random.randn(4,1)*np.sqrt(1/4)
def sigmoid(x):
    y = (1/(1+np.exp(-x)))
    return y

def dersigmoid(x):
    x = np.reshape(np.asarray(x),4)
    y = np.diag(np.divide(exp(x),(1+exp(x))**2))
    return np.asmatrix(y)

def NN(w1,w2,inp,out):
    yest = np.asmatrix(np.zeros((1,100)))
    dif = np.asmatrix(np.zeros((1,100)))
    inpmat1 = np.asmatrix(np.zeros((4,4)))
    inpmat2 = np.asmatrix(np.zeros((4,4)))
    inpmat3 = np.asmatrix(np.zeros((4,4)))
    inpmat4 = np.asmatrix(np.zeros((4,4)))
    dEdw1 = np.asmatrix(np.zeros((4,4)))
    dEdw2 = np.asmatrix(np.zeros((1,4)))
    for i in range(100):
            yest[0,i] = (tp(w2)*sigmoid(w1*tp(inp[i,:]))).item(0)
            dif[0,i] = out[0,i]-yest[0,i]
            inpmat1[0,:] = inp[i,:]
            dEdw1[0,:] = dEdw1[0,:] + (-2*dif[0,i]*(tp(w2)*dersigmoid(w1*tp(inp[i,:]))*inpmat1))
            inpmat2[1,:] = inp[i,:]
            dEdw1[1,:] = dEdw1[1,:] + (-2*dif[0,i]*(tp(w2)*dersigmoid(w1*tp(inp[i,:]))*inpmat2))
            inpmat3[2,:] = inp[i,:] 
            dEdw1[2,:] = dEdw1[2,:] + (-2*dif[0,i]*(tp(w2)*dersigmoid(w1*tp(inp[i,:]))*inpmat3))
            inpmat4[3,:] = inp[i,:]
            dEdw1[3,:] = dEdw1[3,:] + (-2*dif[0,i]*(tp(w2)*dersigmoid(w1*tp(inp[i,:]))*inpmat4))
            
            dEdw2 = dEdw2 + (-2*dif[0,i]*tp(sigmoid(w1*tp(inp[i,:]))))
    MSE = dif*tp(dif) 
    
    
         
    return dEdw1,dEdw2,MSE,yest
        
def Descent(w1,w2,u,y,alpha):
    
    dEdw1,dEdw2,MSE,yest = NN(w1,w2,u,y)
    for i in range(3):
        w1[i,:] = w1[i,:] - alpha*dEdw1[i,:]
    w2 = w2 - alpha*tp(dEdw2)
    
    return w1,w2,MSE,yest

In [ ]:
count = 0
MSE = 1
while MSE > 1e-5:
    w1,w2,MSE,yest = Descent(w1,w2,u,y,1e-4)
    count +=1
    if count % 100 == 0:
        print('Did 100 - MSE : ',MSE)
        

In [ ]:
print(yest[0,10])
print(y[0,10])

In [21]:
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

model = Sequential()


model.add(Dense(4,activation = 'relu',use_bias = 1,input_dim = (4)))
model.add(Dense(1))#activation = 'relu'))
adam = keras.optimizers.Adam(lr=1e-6)
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model.fit(np.reshape(inp,(-1,4)),out, batch_size=100, epochs=20000, verbose=1)
print(history.history.keys())

Epoch 1/20000
100/100 [==============================] - 0s 4ms/step - loss: 1.2309
Epoch 2/20000
100/100 [==============================] - 0s 0us/step - loss: 1.2235
Epoch 3/20000
100/100 [==============================] - 0s 31us/step - loss: 1.2135
Epoch 4/20000
100/100 [==============================] - 0s 18us/step - loss: 1.2036
Epoch 5/20000
100/100 [==============================] - 0s 45us/step - loss: 1.1938
Epoch 6/20000
100/100 [==============================] - 0s 10us/step - loss: 1.1840
Epoch 7/20000
100/100 [==============================] - 0s 42us/step - loss: 1.1743
Epoch 8/20000
100/100 [==============================] - 0s 30us/step - loss: 1.1647
Epoch 9/20000
100/100 [==============================] - 0s 10us/step - loss: 1.1551
Epoch 10/20000
100/100 [==============================] - 0s 0us/step - loss: 1.1455
Epoch 11/20000
100/100 [==============================] - 0s 66us/step - loss: 1.1360
Epoch 12/20000
100/100 [==============================] - 0s 23us/

Epoch 97/20000
100/100 [==============================] - 0s 17us/step - loss: 0.5356
Epoch 98/20000
100/100 [==============================] - 0s 25us/step - loss: 0.5307
Epoch 99/20000
100/100 [==============================] - 0s 20us/step - loss: 0.5258
Epoch 100/20000
100/100 [==============================] - 0s 20us/step - loss: 0.5209
Epoch 101/20000
100/100 [==============================] - 0s 20us/step - loss: 0.5160
Epoch 102/20000
100/100 [==============================] - 0s 14us/step - loss: 0.5112
Epoch 103/20000
100/100 [==============================] - 0s 10us/step - loss: 0.5064
Epoch 104/20000
100/100 [==============================] - 0s 22us/step - loss: 0.5016
Epoch 105/20000
100/100 [==============================] - 0s 19us/step - loss: 0.4969
Epoch 106/20000
100/100 [==============================] - 0s 21us/step - loss: 0.4922
Epoch 107/20000
100/100 [==============================] - 0s 20us/step - loss: 0.4876
Epoch 108/20000
100/100 [=====================

Epoch 192/20000
100/100 [==============================] - 0s 22us/step - loss: 0.1884
Epoch 193/20000
100/100 [==============================] - 0s 26us/step - loss: 0.1863
Epoch 194/20000
100/100 [==============================] - 0s 18us/step - loss: 0.1842
Epoch 195/20000
100/100 [==============================] - 0s 24us/step - loss: 0.1821
Epoch 196/20000
100/100 [==============================] - 0s 23us/step - loss: 0.1801
Epoch 197/20000
100/100 [==============================] - 0s 10us/step - loss: 0.1781
Epoch 198/20000
100/100 [==============================] - 0s 28us/step - loss: 0.1762
Epoch 199/20000
100/100 [==============================] - 0s 17us/step - loss: 0.1743
Epoch 200/20000
100/100 [==============================] - 0s 25us/step - loss: 0.1724
Epoch 201/20000
100/100 [==============================] - 0s 24us/step - loss: 0.1706
Epoch 202/20000
100/100 [==============================] - 0s 16us/step - loss: 0.1688
Epoch 203/20000
100/100 [==================

Epoch 287/20000
100/100 [==============================] - 0s 27us/step - loss: 0.0995
Epoch 288/20000
100/100 [==============================] - 0s 0us/step - loss: 0.0992
Epoch 289/20000
100/100 [==============================] - 0s 31us/step - loss: 0.0988
Epoch 290/20000
100/100 [==============================] - 0s 16us/step - loss: 0.0985
Epoch 291/20000
100/100 [==============================] - 0s 33us/step - loss: 0.0982
Epoch 292/20000
100/100 [==============================] - 0s 32us/step - loss: 0.0979
Epoch 293/20000
100/100 [==============================] - 0s 33us/step - loss: 0.0976
Epoch 294/20000
100/100 [==============================] - 0s 19us/step - loss: 0.0973
Epoch 295/20000
100/100 [==============================] - 0s 23us/step - loss: 0.0970
Epoch 296/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0967
Epoch 297/20000
100/100 [==============================] - 0s 10us/step - loss: 0.0964
Epoch 298/20000
100/100 [===================

Epoch 382/20000
100/100 [==============================] - 0s 19us/step - loss: 0.0766
Epoch 383/20000
100/100 [==============================] - 0s 22us/step - loss: 0.0764
Epoch 384/20000
100/100 [==============================] - 0s 17us/step - loss: 0.0763
Epoch 385/20000
100/100 [==============================] - 0s 19us/step - loss: 0.0761
Epoch 386/20000
100/100 [==============================] - 0s 10us/step - loss: 0.0759
Epoch 387/20000
100/100 [==============================] - 0s 18us/step - loss: 0.0757
Epoch 388/20000
100/100 [==============================] - 0s 35us/step - loss: 0.0755
Epoch 389/20000
100/100 [==============================] - 0s 54us/step - loss: 0.0753
Epoch 390/20000
100/100 [==============================] - 0s 29us/step - loss: 0.0751
Epoch 391/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0749
Epoch 392/20000
100/100 [==============================] - 0s 0us/step - loss: 0.0747
Epoch 393/20000
100/100 [===================

...

Epoch 6585/20000
100/100 [==============================] - 0s 40us/step - loss: 0.0015
Epoch 6586/20000
100/100 [==============================] - 0s 72us/step - loss: 0.0015
Epoch 6587/20000
100/100 [==============================] - 0s 48us/step - loss: 0.0015
Epoch 6588/20000
100/100 [==============================] - 0s 28us/step - loss: 0.0015
Epoch 6589/20000
100/100 [==============================] - 0s 27us/step - loss: 0.0015
Epoch 6590/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6591/20000
100/100 [==============================] - 0s 30us/step - loss: 0.0015
Epoch 6592/20000
100/100 [==============================] - 0s 18us/step - loss: 0.0015
Epoch 6593/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6594/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6595/20000
100/100 [==============================] - 0s 185us/step - loss: 0.0015
Epoch 6596/20000
100/100 [=====

Epoch 6679/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6680/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6681/20000
100/100 [==============================] - 0s 13us/step - loss: 0.0015
Epoch 6682/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6683/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6684/20000
100/100 [==============================] - 0s 15us/step - loss: 0.0015
Epoch 6685/20000
100/100 [==============================] - 0s 21us/step - loss: 0.0015
Epoch 6686/20000
100/100 [==============================] - 0s 22us/step - loss: 0.0015
Epoch 6687/20000
100/100 [==============================] - 0s 29us/step - loss: 0.0015
Epoch 6688/20000
100/100 [==============================] - 0s 39us/step - loss: 0.0015
Epoch 6689/20000
100/100 [==============================] - 0s 37us/step - loss: 0.0015
Epoch 6690/20000
100/100 [======

Epoch 6773/20000
100/100 [==============================] - 0s 30us/step - loss: 0.0015
Epoch 6774/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6775/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6776/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6777/20000
100/100 [==============================] - 0s 22us/step - loss: 0.0015
Epoch 6778/20000
100/100 [==============================] - 0s 23us/step - loss: 0.0015
Epoch 6779/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6780/20000
100/100 [==============================] - 0s 22us/step - loss: 0.0015
Epoch 6781/20000
100/100 [==============================] - 0s 7us/step - loss: 0.0015
Epoch 6782/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6783/20000
100/100 [==============================] - 0s 25us/step - loss: 0.0015
Epoch 6784/20000
100/100 [=======

Epoch 6867/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6868/20000
100/100 [==============================] - 0s 0us/step - loss: 0.0015
Epoch 6869/20000
100/100 [==============================] - 0s 60us/step - loss: 0.0015
Epoch 6870/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6871/20000
100/100 [==============================] - 0s 23us/step - loss: 0.0015
Epoch 6872/20000
100/100 [==============================] - 0s 19us/step - loss: 0.0015
Epoch 6873/20000
100/100 [==============================] - 0s 10us/step - loss: 0.0015
Epoch 6874/20000
100/100 [==============================] - 0s 38us/step - loss: 0.0015
Epoch 6875/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6876/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6877/20000
100/100 [==============================] - 0s 20us/step - loss: 0.0015
Epoch 6878/20000
100/100 [=======

KeyboardInterrupt: 

In [ ]:
a = 5
print(model.predict(np.reshape(inp[a],(1,-1))));
print(out[a])

In [ ]:
(np.asarray(u)).reshape((100,1,4))

In [ ]:
MSE = 1

In [ ]:
count

In [ ]:
a,b,c,yest = NN(w1,w2,u,y)
y

In [ ]:
w1

In [ ]:
   for i in range(len(classlist)):
        layerinput = l[i]
        ichain = chain[i]
        [row_no,column_no] = classlist[i].mat.shape
        dmat = np.zeros((row_no,column_no+1))
        for j in range(column_no+1):
            summ = np.zeros((1,row_no))
            for k in range(inp.shape[0]):
                if j != column_no:
                    inp1 = layerinput[k]
                    id_input = (np.identity(row_no)*inp1[j])
                else:
                    inp1 = layerinput[k]
                    id_input = (np.identity(row_no))
                dodj = ichain@id_input                      
                dedj = (-2*dif[k]*dodj)
                summ += dedj
                
            dmat[:,j] = summ
        storage.append(dmat)
    
    return storage

def LayerInputs(classlist,inp):
    list_in = [inp]
    
    for i in range(len(classlist)):
        depo = []
        for k in inp:
            out = classlist[i].LayerOut(k)
            depo.append(out)
        depo = np.asarray(depo)
        inp = depo
        list_in.append(depo)
    return list_in